In [1]:
import logging
logging.basicConfig(level = logging.INFO)

import os
import torch
from collections import OrderedDict
from torch.nn.parallel import DistributedDataParallel

In [2]:
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer, PeriodicCheckpointer
from detectron2.config import get_cfg
from detectron2.data import (
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.engine import default_argument_parser, default_setup, launch
from detectron2.evaluation import (
    CityscapesEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    inference_on_dataset,
    print_csv_format,
)
from detectron2.modeling import build_model
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.utils.events import (
    CommonMetricPrinter,
    EventStorage,
    JSONWriter,
    TensorboardXWriter,
)

logging.basicConfig(level = logging.INFO)
logger = logging.getLogger("detectron2")

def setup(config_file):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    # cfg.merge_from_file(args.config_file)
    cfg.merge_from_file(config_file)
    # cfg.merge_from_list(args.opts)
    
    
    # cfg.freeze() # make it immutable
    # default_setup(
    #    cfg, args
    # )  
    # if you don't like any of the default setup, 
    # write your own setup code
    return cfg

In [3]:
def do_train(cfg, model, resume=False):
    print("do_train")
    model.train()
    optimizer = build_optimizer(cfg, model)
    scheduler = build_lr_scheduler(cfg, optimizer)

    checkpointer = DetectionCheckpointer(
        model, cfg.OUTPUT_DIR, optimizer=optimizer, scheduler=scheduler
    )
    start_iter = (
        checkpointer.resume_or_load(cfg.MODEL.WEIGHTS, resume=resume).get("iteration", -1) + 1
    )
    max_iter = cfg.SOLVER.MAX_ITER

    periodic_checkpointer = PeriodicCheckpointer(
        checkpointer, cfg.SOLVER.CHECKPOINT_PERIOD, max_iter=max_iter
    )

    writers = (
        [
            CommonMetricPrinter(max_iter),
            JSONWriter(os.path.join(cfg.OUTPUT_DIR, "metrics.json")),
            TensorboardXWriter(cfg.OUTPUT_DIR),
        ]
        if comm.is_main_process()
        else []
    )

    # compared to "train_net.py", we do not support accurate timing and
    # precise BN here, because they are not trivial to implement
    data_loader = build_detection_train_loader(cfg)
    logger.info("Starting training from iteration {}".format(start_iter))
    with EventStorage(start_iter) as storage:
        for data, iteration in zip(data_loader, range(start_iter, max_iter)):
            iteration = iteration + 1
            storage.step()

            loss_dict = model(data)
            losses = sum(loss_dict.values())
            assert torch.isfinite(losses).all(), loss_dict

            loss_dict_reduced = {k: v.item() for k, v in comm.reduce_dict(loss_dict).items()}
            losses_reduced = sum(loss for loss in loss_dict_reduced.values())
            if comm.is_main_process():
                storage.put_scalars(total_loss=losses_reduced, **loss_dict_reduced)

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            storage.put_scalar("lr", optimizer.param_groups[0]["lr"], smoothing_hint=False)
            scheduler.step()

            if (
                cfg.TEST.EVAL_PERIOD > 0
                and iteration % cfg.TEST.EVAL_PERIOD == 0
                and iteration != max_iter
            ):
                do_test(cfg, model)
                # Compared to "train_net.py", the test results are not dumped to EventStorage
                comm.synchronize()

            if iteration - start_iter > 5 and (iteration % 20 == 0 or iteration == max_iter):
                for writer in writers:
                    writer.write()
            periodic_checkpointer.step(iteration)

### Get config

In [9]:
cfg = setup("dt2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")

In [6]:
from detectron2.data.datasets import register_coco_instances

COCO = "/home/quang/datasets/COCO_2017/coco"
register_coco_instances("my_train", {}, 
                        os.path.join(COCO, "annotations/instances_train2017.json"),
                        os.path.join(COCO, "train2017")
                       )
                                     
register_coco_instances("my_val", {}, 
                        os.path.join(COCO, "annotations/instances_val2017.json"),
                        os.path.join(COCO, "val2017")
                       )

cfg.DATASETS.TRAIN = ("my_train", )
cfg.DATASETS.TEST = ("my_val", )
cfg.SOLVER.MAX_ITER = 10

In [7]:
model = build_model(cfg)

In [10]:
model

GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2): Sequential(
        (0): BottleneckBlock

In [11]:
run dist_launch.py

In [12]:
distributed = comm.get_world_size() > 1
if distributed:
    model = DistributedDataParallel(
        model, device_ids=[comm.get_local_rank()], broadcast_buffers=False
    )


In [16]:
import torch.distributed as dist

comm.get_local_rank()

0

In [19]:
dist.init_process_group?

Signature:
dist.init_process_group(
    backend,
    init_method=None,
    timeout=datetime.timedelta(seconds=1800),
    world_size=-1,
    rank=-1,
    store=None,
    group_name='',
)
Docstring:
Initializes the default distributed process group, and this will also
initialize the distributed package.

There are 2 main ways to initialize a process group:
    1. Specify ``store``, ``rank``, and ``world_size`` explicitly.
    2. Specify ``init_method`` (a URL string) which indicates where/how
       to discover peers. Optionally specify ``rank`` and ``world_size``,
       or encode all required parameters in the URL and omit them.

If neither is specified, ``init_method`` is assumed to be "env://".


Arguments:
    backend (str or Backend): The backend to use. Depending on
        build-time configurations, valid values include ``mpi``, ``gloo``,
        and ``nccl``. This field should be given as a lowercase string
        (e.g., ``"gloo"``), which can also be accessed via
        :clas